# Cosmin Madalin Zaharia

### Implementation of findpeak and meanshift not optimazed

<br>

In [4]:
from scipy.spatial.distance import cdist
import scipy.io
import numpy as np

<br>

According to the homework: define a <i><b>findpeak</b></i> Where data is the n-dimensional dataset consisting of p points, idx is the column index of the data point for which we wish to compute its associated density peak, and r is the search window radius.

In [8]:
def findpeak(data, idx, r, tau = 0.01): #in for a faster speed you can decrease tau
        
        #peak the vector along the column
        point = data[:,idx]
        
        #calculate the distance between the dataset and the point
        distance = cdist(data.T,point.reshape(1,-1))
        
        #define the window size
        window_size = np.where(distance < r)[0]
        
        #Now let's calculate the mean
        #Since I want to get a (3,) vector, the axis = 1
        mean_points = np.mean(data[:,window_size], axis=1) 
        
        #simple euclidian distance and repeat the process above until the convergece
        while (np.linalg.norm(point - mean_points)) > tau:
            point = mean_points
            distance = cdist(data.T,point.reshape(1,-1))
            window_size = np.where(distance < r)[0]
            mean_points = np.mean(data[:,window_size], axis=1)
        
        return point

<br>

According to the homework: define a <i><b>meanshift</b></i> function where peaks are compared after each call to the findpeak function and for similar peaks to be merged. For our implementation of meanshift, we will consider two peaks to be the same if the distance between them is smaller than r/2. Also, if the peak of a data point is found to already exist in peaks then for simplicity its computed peak is discarded and it is given the label of the associated peak in peaks.


In [9]:
def meanshift(data, r):
    #define the label 
    labels = np.zeros(data.shape[1])-1 
    peaks = []
    
    #first iteration
    peak = findpeak(data, 0 , r)
    labels[0] = 0
    peaks.append(peak)
    

    for idx in range(1, data.shape[1]):
        peak = findpeak(data, idx, r)
        
        #after you got the peak calculate the distance with the other peaks and see if it is smaller than r<2
        distance=np.linalg.norm(peaks - peak, axis = 1)
        similar_peaks = np.argwhere(distance < r/2)
        
        if len(similar_peaks) > 0:
            labels[idx] = similar_peaks[0]
        else:
            #this means that we aren't the window so the peak doesn't belong to that cluster => we found a new cluster
            peaks.append(peak)
            labels[idx] = len(peaks)

    return labels, peaks

<br>

Test:

In [10]:
mat = scipy.io.loadmat('pts.mat')
meanshift(mat["data"] , 2)

(array([0., 0., 0., ..., 1., 1., 1.]),
 [array([-0.06756875,  0.04788814,  0.02778451]),
  array([5.04987186, 4.98116882, 5.0104074 ])])